In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=7142b9aed5a1622e28f44243ee1bf5eeb6b1a95f51eb8f41c4fd7ba864be6a7e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [2]:
import numpy as np
import pandas as pd

from string import digits
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
from tqdm import tqdm, notebook

from sentence_transformers import SentenceTransformer, CrossEncoder, util
import torch

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/kaggle/input/youtube-videos-title-description-comments/GBvideos.csv
/kaggle/input/youtube-videos-title-description-comments/GBtext-details.csv
/kaggle/input/youtube-videos-title-description-comments/UScomments.csv
/kaggle/input/youtube-videos-title-description-comments/GB_category_id.json
/kaggle/input/youtube-videos-title-description-comments/US_category_id.json
/kaggle/input/youtube-videos-title-description-comments/UStext-details.csv
/kaggle/input/youtube-videos-title-description-comments/GBcomments.csv
/kaggle/input/youtube-videos-title-description-comments/USvideos.csv


### Reading Data

Reading GB data

In [3]:
gb_detail_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/GBvideos.csv', on_bad_lines='skip')
gb_detail_df.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Apple Event,28,apple events|apple event|iphone 8|iphone x|iph...,7426393,78240,13548,705,https://i.ytimg.com/vi/jt2OHQh0HoQ/default_liv...,13.09
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,This Morning,24,this morning|interview|holly willoughby|philli...,494203,2651,1309,0,https://i.ytimg.com/vi/AqokkXoa7uE/default.jpg,13.09
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,emmablackery,24,emmablackery|emma blackery|emma|blackery|briti...,142819,13119,151,1141,https://i.ytimg.com/vi/YPVcg45W0z4/default.jpg,13.09
3,T_PuZBdT2iM,getting into a conversation in a language you ...,ProZD,1,skit|korean|language|conversation|esl|japanese...,1580028,65729,1529,3598,https://i.ytimg.com/vi/T_PuZBdT2iM/default.jpg,13.09
4,NsjsmgmbCfc,Baby Name Challenge!,Sprinkleofglitter,26,sprinkleofglitter|sprinkle of glitter|baby gli...,40592,5019,57,490,https://i.ytimg.com/vi/NsjsmgmbCfc/default.jpg,13.09


In [4]:
gb_text_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/GBtext-details.csv')
gb_text_df = gb_text_df.drop('Unnamed: 0', axis=1)
gb_text_df.head()

,id,title,description
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Live Apple Event - Apple September Event 2017 ...
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,Subscribe now for more! http://bit.ly/1JM41yF\...
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,extra vlogs: http://youtube.com/c/vloggery \ni...
3,T_PuZBdT2iM,getting into a conversation in a language you ...,My Twitter: https://twitter.com/prozd\nMy Let'...
4,NsjsmgmbCfc,Crazy Baby Names! | MOTHERHOOD,"All things Names! Cute, weird, wonderful baby ..."


In [5]:
gb_comments_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/GBcomments.csv', on_bad_lines='skip')
gb_comments_df.head()

,video_id,comment_text,likes,replies
0,jt2OHQh0HoQ,It's more accurate to call it the M+ (1000) be...,0,0
1,jt2OHQh0HoQ,To be there with a samsung phone\n😂😂😂,1,0
2,jt2OHQh0HoQ,"Thank gosh, a place I can watch it without hav...",0,0
3,jt2OHQh0HoQ,What happened to the home button on the iPhone...,0,0
4,jt2OHQh0HoQ,Power is the disease. Care is the cure. Keep...,0,0


Reading us data

In [6]:
us_detail_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/USvideos.csv', on_bad_lines='skip')
us_detail_df.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09


In [7]:
us_text_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/UStext-details.csv')
us_text_df = us_text_df.drop('Unnamed: 0', axis=1)
us_text_df.head()

,id,title,description
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Grab your 365 DAY MAVERICK MERCH! Exclusively ...
1,cLdxuaxaQwc,My Response,NaN
2,WYYvHb03Eog,Apple iPhone X first look,The Apple iPhone X is the new super flagship p...
3,sjlHnJvXdQs,iPhone X (parody),MORE APPLE STUFF ► https://www.youtube.com/wat...
4,cMKX2tE5Luk,The Disaster Artist | Official Trailer HD | A24,SUBSCRIBE: http://bit.ly/A24subscribe\n\nBased...


Concatenating the GB and US data

In [8]:
us_comments_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/UScomments.csv', on_bad_lines='skip')
us_comments_df.head()

/tmp/ipykernel_31/3472227992.py:1: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  us_comments_df = pd.read_csv('/kaggle/input/youtube-videos-title-description-comments/UScomments.csv', on_bad_lines='skip')


,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0
4,XpVt6Z1Gjjo,trending 😉,3,0


In [9]:
text_df = pd.concat([gb_text_df, us_text_df], axis=0)
print("Len of GB data - ", len(gb_text_df))
print("Len of GB data - ", len(us_text_df))
print("Len of combined data - ", len(text_df))
text_df.head()

Len of GB data -  7120
Len of GB data -  7227
Len of combined data -  14347


,id,title,description
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Live Apple Event - Apple September Event 2017 ...
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,Subscribe now for more! http://bit.ly/1JM41yF\...
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,extra vlogs: http://youtube.com/c/vloggery \ni...
3,T_PuZBdT2iM,getting into a conversation in a language you ...,My Twitter: https://twitter.com/prozd\nMy Let'...
4,NsjsmgmbCfc,Crazy Baby Names! | MOTHERHOOD,"All things Names! Cute, weird, wonderful baby ..."


In [10]:
comments_df = pd.concat([gb_comments_df, us_comments_df], axis=0)
print("Len of GB data - ", len(gb_comments_df))
print("Len of GB data - ", len(us_comments_df))
print("Len of combined data - ", len(comments_df))
comments_df.head()

Len of GB data -  718452
Len of GB data -  691400
Len of combined data -  1409852


,video_id,comment_text,likes,replies
0,jt2OHQh0HoQ,It's more accurate to call it the M+ (1000) be...,0,0
1,jt2OHQh0HoQ,To be there with a samsung phone\n😂😂😂,1,0
2,jt2OHQh0HoQ,"Thank gosh, a place I can watch it without hav...",0,0
3,jt2OHQh0HoQ,What happened to the home button on the iPhone...,0,0
4,jt2OHQh0HoQ,Power is the disease. Care is the cure. Keep...,0,0


In [11]:
details_df = pd.concat([gb_detail_df, us_detail_df], axis=0)
print("Len of GB data - ", len(gb_detail_df))
print("Len of GB data - ", len(us_detail_df))
print("Len of combined data - ", len(details_df))
details_df.head()

Len of GB data -  7993
Len of GB data -  7992
Len of combined data -  15985


,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Apple Event,28,apple events|apple event|iphone 8|iphone x|iph...,7426393,78240,13548,705,https://i.ytimg.com/vi/jt2OHQh0HoQ/default_liv...,13.09
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,This Morning,24,this morning|interview|holly willoughby|philli...,494203,2651,1309,0,https://i.ytimg.com/vi/AqokkXoa7uE/default.jpg,13.09
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,emmablackery,24,emmablackery|emma blackery|emma|blackery|briti...,142819,13119,151,1141,https://i.ytimg.com/vi/YPVcg45W0z4/default.jpg,13.09
3,T_PuZBdT2iM,getting into a conversation in a language you ...,ProZD,1,skit|korean|language|conversation|esl|japanese...,1580028,65729,1529,3598,https://i.ytimg.com/vi/T_PuZBdT2iM/default.jpg,13.09
4,NsjsmgmbCfc,Baby Name Challenge!,Sprinkleofglitter,26,sprinkleofglitter|sprinkle of glitter|baby gli...,40592,5019,57,490,https://i.ytimg.com/vi/NsjsmgmbCfc/default.jpg,13.09


In [12]:
device = 'cuda'
if torch.cuda.is_available():      
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


Looking at data for a particular video

In [13]:
text_df.rename(columns = {'id':'video_id'}, inplace = True)

In [14]:
text_df['video_id'].sample(10)

2599    jizvSedEDDI
6543    vUC8-qbCVWk
3225    Hg6L_7qLIEQ
1170    gIE105f0ejg
5643    aDuB1arf8aE
549     nnTcCS07DKQ
5056    ha-OKyNiHNE
5756    mi52IqpOp54
1572    S7rQSRGGKSQ
6731    xLZoYh-aMm8
Name: video_id, dtype: object

Title

In [15]:
text_df.iloc[2176].title

"ULTIMATE GYMNASTICS FAILS | Who's in the Worlds Team?"

Description

In [16]:
text_df.iloc[2176].description

"➢ Join my FREE 7 Steps to Success - http://www.bodybible.life/\n➢ Merch- http://www.nwclothing.co.uk/\n➢ Enquiries - luke@activatemanagement.co.uk\n\n➢ Handstand Programme 🤸🏼\u200d♂️https://bodybible.life/individual-programmes-2/\n➢ Depop -https://www.depop.com/joannaj\n➢ Gymnastics Gear ~ https://www.turn-gymnastics.com/\n➢ Intro Song ~ Tom Zanetti, Darlin'\n➢ Soundcloud ~ https://soundcloud.com/nile-wilson-2/sets/wobblers\n➢Instagram https://instagram.com/nilemw/\n➢ Body Bible instagram https://www.instagram.com/nw_bodybible/\n➢Twitter https://twitter.com/NileMW\n➢Facebook https://www.facebook.com/NileWilsonGy…\n➢SnapChat - nilewilson\n➢Songs ~ https://www.youtube.com/user/officialdjwelshy"

Comments

In [17]:
video_comments = comments_df[comments_df['video_id'] == '7ocf8suHnP0'].comment_text.values
print("Total comments for video - 7ocf8suHnP0 = ", len(video_comments))
print("Channel title for video - 7ocf8suHnP0 = ", details_df[details_df['video_id'] == '7ocf8suHnP0'].channel_title)
video_comments[:5]

Total comments for video - 7ocf8suHnP0 =  600
Channel title for video - 7ocf8suHnP0 =  2030    Nile Wilson
2244    Nile Wilson
2450    Nile Wilson
2663    Nile Wilson
2935    Nile Wilson
3181    Nile Wilson
Name: channel_title, dtype: object


array(["James x'D especially when he gives up in the first clip hahah, but gottta admit Danny's fail is hard to\xa0top loll, poor him!!",
       'James for the fail / win made me 😂 and congrats for making the team keep up the hard work',
       "The second one hitting the bars in the worst place possible 😂 nile's videos are my daily dose of good looking, charming men that are painfully absent in my own life",
       'Nile, cringiest: Your fall on the high bar.  It echoed thru the building.  Funniest:  the Purvis, definitely! Love the video!',
       'Interesting.. I always wondered what those hand guard things were.'],
      dtype=object)

In [18]:
def clean_text(text, remove_stopwords=True):
    text = text.lower()
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words('english'))
        text = [w for w in text if w not in stops]
        text = ' '.join(text)
        
    return text

In [19]:
def clean_data(data):
    cleaned_data = []
    for doc in notebook.tqdm(data):
        text = clean_text(doc, False)
        cleaned_data.append(text)
    return cleaned_data

In [20]:
clean_data([text_df.iloc[2176].description])

  0%|          | 0/1 [00:00<?, ?it/s]

['➢ join my free 7 steps to success   ➢ merch  ➢ enquiries   luke activatemanagement co uk\n\n➢ handstand programme 🤸🏼\u200d♂️➢ depop  ➢ gymnastics gear ~ ➢ intro song ~ tom zanetti  darlin \n➢ soundcloud ~ ➢instagram ➢ body bible instagram ➢twitter ➢facebook ➢snapchat   nilewilson\n➢songs ~ ']

In [21]:
from nltk.tokenize import RegexpTokenizer
import re
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    return " ".join(filtered_words)

preprocess(text_df.iloc[2176].description)

'join free steps success merch enquiries luke activatemanagement handstand programme depop gymnastics gear intro song tom zanetti darlin soundcloud instagram body bible instagram twitter facebook snapchat nilewilson songs'

In [22]:
embedder = SentenceTransformer('all-MiniLM-L6-v2', device=device)
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', device=device)

We are joining video title, channel title and description of the video to judge relevancy. As much of the text is there which relates to the video content, it'll be better to judge relevancy using the sentence embeddings.

In [23]:
cleaned_description = preprocess(text_df.iloc[2176].description)
title = text_df.iloc[2176].title
channel_title = details_df[details_df['video_id'] == '7ocf8suHnP0'].channel_title.values[0]
query_text = title + '. ' + cleaned_description + '. ' + channel_title
query_text

"ULTIMATE GYMNASTICS FAILS | Who's in the Worlds Team?. join free steps success merch enquiries luke activatemanagement handstand programme depop gymnastics gear intro song tom zanetti darlin soundcloud instagram body bible instagram twitter facebook snapchat nilewilson songs. Nile Wilson"

In [24]:
comments = comments_df[comments_df['video_id'] == '7ocf8suHnP0'].comment_text.values
comments[0]

"James x'D especially when he gives up in the first clip hahah, but gottta admit Danny's fail is hard to\xa0top loll, poor him!!"

In [25]:
len(set(comments))

149

Only 149 unique comments.

In [26]:
comments = list(set(comments))

In [27]:
def get_embeddings(data):
    return embedder.encode(data, convert_to_tensor=True)

In [28]:
comments_embeddings = get_embeddings(comments)
comments_embeddings.shape

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

torch.Size([149, 384])

In [29]:
query_embedding = get_embeddings([query_text])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
hits = util.semantic_search(query_embedding, comments_embeddings, top_k=149)
hits = hits[0]

Top 10 comments by relevancy

In [31]:
hits[:10]

[{'corpus_id': 132, 'score': 0.5700268745422363},
 {'corpus_id': 45, 'score': 0.4984581768512726},
 {'corpus_id': 89, 'score': 0.4978342652320862},
 {'corpus_id': 12, 'score': 0.4903081953525543},
 {'corpus_id': 35, 'score': 0.4816342890262604},
 {'corpus_id': 40, 'score': 0.47107747197151184},
 {'corpus_id': 7, 'score': 0.46166250109672546},
 {'corpus_id': 71, 'score': 0.432467520236969},
 {'corpus_id': 102, 'score': 0.39275628328323364},
 {'corpus_id': 106, 'score': 0.36715829372406006}]

In [32]:
for hit in hits[:10]:
    print("Comment - ", comments[hit['corpus_id']])

Comment -  What happened to the ultimate gymnastics challenge??
Comment -  you should make a video of trying girls gymnastics
Comment -  you motivate me to keep on pushing at gymnastics I'm 12 and aspire to be as good as you one day Nile :)
Comment -  talk to plan zero about doing a kind of tricking vs gymnastics video
Comment -  I just started gymnastics and it turns out I’m actually pretty good and part of why I love gymnastics is because you fail a lot and I used to play other sports where I always did well and never really reached my limit but with gymnastics I feel I have to work really hard for everything and it makes me happy I really hope one day I get to represent my country
Comment -  Tbh I don’t even do gymnastics I play soccer but I love ur video because of ur positivity
Comment -  Purvis! Amazing fail. Awesome gymnast 😀 (you too Nile! Congrats!!!)
Comment -  Purvis just wins this shit hands down for falling into the judges. Such a beautiful moment in gymnastics, lol.
Comme

Bottom 10 comments. Least relevant according to model

In [33]:
hits[-10:]

[{'corpus_id': 75, 'score': 0.0336839035153389},
 {'corpus_id': 144, 'score': 0.026752430945634842},
 {'corpus_id': 123, 'score': 0.023879840970039368},
 {'corpus_id': 25, 'score': 0.016111791133880615},
 {'corpus_id': 31, 'score': 0.00827944464981556},
 {'corpus_id': 54, 'score': 0.005353173241019249},
 {'corpus_id': 98, 'score': 0.0011678291484713554},
 {'corpus_id': 49, 'score': -0.002962414175271988},
 {'corpus_id': 36, 'score': -0.013926508836448193},
 {'corpus_id': 116, 'score': -0.02331463433802128}]

In [34]:
for hit in hits[-10:]:
    print("Comment - ", comments[hit['corpus_id']])

Comment -  Definitely the ginger one
Comment -  Purvis!!!!
Comment -  The first and purvis
Comment -  Purvis definitely
Comment -  Ginger guy is jokes
Comment -  Stereograms of Wealth. \nMatrix of Wealth. \nhttps://www.etsy.com/listing/550842299\nBuy One Stereogram.\nGet 3 Stereograms in Gift.\nStereograms to order.\nPersonalized stereograms.
Comment -  5.41 ur face
Comment -  Purvis
Comment -  Hope it's not too late for votes, mine definitly goes to Purvis
Comment -  I just go harassed by kids at a park


Comments are less. We can directly try cross-encoder for all comments.

In [35]:
cross_inp = [[query_text, comment] for comment in comments]

In [36]:
cross_inp[0]

["ULTIMATE GYMNASTICS FAILS | Who's in the Worlds Team?. join free steps success merch enquiries luke activatemanagement handstand programme depop gymnastics gear intro song tom zanetti darlin soundcloud instagram body bible instagram twitter facebook snapchat nilewilson songs. Nile Wilson",
 'Hi']

In [37]:
cross_scores = cross_encoder.predict(cross_inp)

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

In [38]:
cross_scores[:10]

array([-10.28525 , -10.435902, -10.74368 ,  -9.759258,  -8.514727,
        -8.896858, -10.310989,  -7.465781, -10.480262, -10.319725],
      dtype=float32)

In [39]:
for idx in range(len(cross_scores)):
    hits[idx]['cross-score'] = cross_scores[idx]
    
cross_score_sorted_hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
cross_score_sorted_hits[:5]

[{'corpus_id': 115, 'score': 0.05573821812868118, 'cross-score': -6.089186},
 {'corpus_id': 29, 'score': 0.1199600100517273, 'cross-score': -6.5821266},
 {'corpus_id': 71, 'score': 0.432467520236969, 'cross-score': -7.465781},
 {'corpus_id': 118, 'score': 0.28523868322372437, 'cross-score': -7.5717106},
 {'corpus_id': 43, 'score': 0.20508168637752533, 'cross-score': -8.114829}]

In [40]:
hits[:5]

[{'corpus_id': 132, 'score': 0.5700268745422363, 'cross-score': -10.28525},
 {'corpus_id': 45, 'score': 0.4984581768512726, 'cross-score': -10.435902},
 {'corpus_id': 89, 'score': 0.4978342652320862, 'cross-score': -10.74368},
 {'corpus_id': 12, 'score': 0.4903081953525543, 'cross-score': -9.759258},
 {'corpus_id': 35, 'score': 0.4816342890262604, 'cross-score': -8.514727}]

No overlap in the top 5 hits between cross encoder and sentence transformer.

First 10 hits from cross encoder.

In [41]:
for hit in cross_score_sorted_hits[:10]:
    print("Comment - ", comments[hit['corpus_id']])

Comment -  #voteJames
Comment -  Never clicked a video so fast
Comment -  Purvis just wins this shit hands down for falling into the judges. Such a beautiful moment in gymnastics, lol.
Comment -  Loved these fails 😃 I think my winner is James.
Comment -  The second one hitting the bars in the worst place possible 😂 nile's videos are my daily dose of good looking, charming men that are painfully absent in my own life
Comment -  how do you get into the olympics
Comment -  I just started gymnastics and it turns out I’m actually pretty good and part of why I love gymnastics is because you fail a lot and I used to play other sports where I always did well and never really reached my limit but with gymnastics I feel I have to work really hard for everything and it makes me happy I really hope one day I get to represent my country
Comment -  10 on trending Nile
Comment -  looks so painful!
Comment -  Purvis!!!!


Last 10 hits from cross encoder.

In [42]:
cross_score_sorted_hits[-10:]

[{'corpus_id': 52, 'score': 0.07161518186330795, 'cross-score': -10.721422},
 {'corpus_id': 89, 'score': 0.4978342652320862, 'cross-score': -10.74368},
 {'corpus_id': 49, 'score': -0.002962414175271988, 'cross-score': -10.74564},
 {'corpus_id': 16, 'score': 0.22971954941749573, 'cross-score': -10.761129},
 {'corpus_id': 14, 'score': 0.18436603248119354, 'cross-score': -10.798369},
 {'corpus_id': 138, 'score': 0.07779933512210846, 'cross-score': -10.852501},
 {'corpus_id': 60, 'score': 0.18286964297294617, 'cross-score': -10.924787},
 {'corpus_id': 97, 'score': 0.18579351902008057, 'cross-score': -10.963195},
 {'corpus_id': 20, 'score': 0.2154240906238556, 'cross-score': -11.125592},
 {'corpus_id': 61, 'score': 0.1911226361989975, 'cross-score': -11.225331}]

In [43]:
for hit in cross_score_sorted_hits[-10:]:
    print("Comment - ", comments[hit['corpus_id']])

Comment -  Lmao 😂😂😂 I vote James , its was a fn good laugh
Comment -  you motivate me to keep on pushing at gymnastics I'm 12 and aspire to be as good as you one day Nile :)
Comment -  Purvis
Comment -  I reckon James had the best fails.
Comment -  I mean Dan's gotta win but James' are for sure up there😂
Comment -  Omg I live in canada
Comment -  the second guy😂😂😂
Comment -  Nile Purvis and the second guy
Comment -  Falling on top of the jugdes has got to be my favorite one lmao
Comment -  Danny just took the cake🤣😂🤣
